<a href="https://colab.research.google.com/github/MatCat776/InterviewBot/blob/main/interview_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My Job Interview Bot


## Configuration

### Install Libraries

In [ ]:
%pip install --upgrade --quiet google-genai pandas==2.2.2
# not sure I really need pandas, colab appears to only support old pandas


In [ ]:
import base64
from time import sleep

### Configure Gemini

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata

# create a google api key and save it as a secret in colab before running next
# line
GOOGLE_API_KEY = userdata.get('AI_STUDIO_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)
model_name = "gemini-2.5-flash-lite"


### Configure google drive

In [ ]:
from google.colab import drive
# next line requires granting permissions to access drive
drive.mount('/content/drive')

## Demo for multi-turn chat

In [ ]:
from google.generativeai import configure
from google.generativeai import GenerativeModel
configure(api_key=GOOGLE_API_KEY)
my_instructions='''Provide short answers to each question.'''
#TODO: add safety_settings as input to next line
model = GenerativeModel(model_name=model_name, system_instruction=my_instructions)

chat = model.start_chat(history=[])
user_message_1 = "Hello, what is the capital of France?"
response_1 = chat.send_message(user_message_1)
print(f"User: {user_message_1}")
print(f"Gemini: {response_1.text}")
user_message_2 = "Hello, what the largest airport is in that city?"
response_2 = chat.send_message(user_message_2)
print(f"User: {user_message_2}")
print(f"Gemini: {response_2.text}")

Multi-turn Chat with job posting

In [ ]:
# Read the content of the text file
with open("/content/drive/My Drive/job_posting.txt", "r") as f:
    job_posting_content = f.read()

chat = model.start_chat(history=[])
user_message_1 = "Here is a copy of the job posting: " + job_posting_content + "What is the job title?"
response_1 = chat.send_message(user_message_1)
print(f"User: {user_message_1}")
print(f"Gemini: {response_1.text}")
user_message_2 = "What is the salary range?"
response_2 = chat.send_message(user_message_2)
print(f"User: {user_message_2}")
print(f"Gemini: {response_2.text}")

## Read Job Posting, text method

In [ ]:
# # Read the content of the text file
# with open("/content/drive/My Drive/job_posting.txt", "r") as f:
#     job_posting_content = f.read()

# contents = [
#     types.Content(
#       role="user",
#       parts=[
#         types.Part.from_text(text="""Tell me about the document"""),
#         types.Part.from_text(text=job_posting_content), # Include the text content directly
#         types.Part.from_text(text="""Say Boo when done""")
#       ]
#     )
#   ]

# # # Response as a stream
# #for chunk in client.models.generate_content_stream(
# #    model = model_name,
# #    contents = contents,
# #    config = generate_content_config,
# #    ):
# #    print(chunk.text, end="")

# # Respond as a single answer
# response = client.models.generate_content(
#     model=model_name, contents=contents
# )
# print(response.text)

# # next answer doesn't have job posting available to it.
# # TODO: do I need to change to a continuous chat format?
# response = client.models.generate_content(
#     model=model_name, contents="""what is the salary range?"""
# )

# print(response.text)

## Questions that use my resume, PDF method.
Pulling content from google drive and github doesn't seem to work in colab the same way it does in Virtex AI. For some reason, I have to convert the pdf to txt, then I can use it here.

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber

def convert_pdf_to_txt_pdfplumber(pdf_path, txt_path):
    """Converts a PDF file to a plain text file using pdfplumber."""
    try:
        with pdfplumber.open(pdf_path) as pdf, open(txt_path, "w", encoding="utf-8") as f:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    f.write(text + '\n')  # Add a newline to separate pages
        print(f"Text successfully extracted from {pdf_path} to {txt_path}")
    except Exception as e:
        print(f"Error converting PDF: {e}")

# Example usage:
# convert_pdf_to_txt_pdfplumber("input.pdf", "output.txt")

def get_text_from_pdf(pdf_path):
  convert_pdf_to_txt_pdfplumber("/content/drive/My Drive/My_Resume.pdf", "/content/drive/My Drive/temp.txt")
  with open("/content/drive/My Drive/temp.txt", "r") as f:
    text_resume_content = f.read()
    return text_resume_content

In [ ]:
# Multi turn chat method
text_resume_content = get_text_from_pdf("/content/drive/My Drive/resume_as_txt.txt")

user_message = "Here is a copy of my resume: " + text_resume_content + "Am I a good fit for this job?"
response = chat.send_message(user_message)
print(f"User: {user_message}")
print(f"Gemini: {response.text}")


In [ ]:
# # Single prompt question method
# text_resume_content = get_text_from_pdf("/content/drive/My Drive/resume_as_txt.txt")
# # Don't know why, but printing the resume content is required
# #   maybe I could do some other interaction with the variable instead?
# #   sleep doesn't work, but print does
# print(text_resume_content)
# print("end of resume content")

# contents = [
#     types.Content(
#       role="user",
#       parts=[
#         types.Part.from_text(text="""Tell me about the document"""),
#         types.Part.from_text(text=text_resume_content), # Include the text content directly
#         types.Part.from_text(text="""Say Boo PDF when done""")
#       ]
#     )
#   ]



# for chunk in client.models.generate_content_stream(
#     model = model_name,
#     contents = contents,
#     ):
#     print(chunk.text, end="")

## Static model settings

In [ ]:
#TODO: move resume import here
my_instructions = """
You are a job applicant in a job interview. This is your resume:
""" + text_resume_content + """ This is the job posting: """ + job_posting_content

generate_content_config = types.GenerateContentConfig(
    temperature = .5,
    top_p = 0.95,
    max_output_tokens = 2048,
    safety_settings = [types.SafetySetting(
      category="HARM_CATEGORY_HATE_SPEECH",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_DANGEROUS_CONTENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
      threshold="BLOCK_LOW_AND_ABOVE"
    ),types.SafetySetting(
      category="HARM_CATEGORY_HARASSMENT",
      threshold="BLOCK_LOW_AND_ABOVE"
    )],
    system_instruction=[types.Part.from_text(text=my_instructions)],
    thinking_config=types.ThinkingConfig(
      thinking_budget=0,
    ),
  )

## Start up model

In [ ]:
#TODO: add safety_settings as input to next line
model = GenerativeModel(model_name=model_name, system_instruction=my_instructions)

chat = model.start_chat(history=[])

def ask_question(question):
  user_message = question
  response = chat.send_message(user_message)
  print(f"User: {user_message}")
  print(f"Gemini: {response.text}")

## Interview Questions

In [ ]:
user_message = "What is your name?"
response = chat.send_message(user_message)
print(f"User: {user_message}")
print(f"Gemini: {response.text}")

ask_question("Why are you a good fit for this job?")
ask_question("What is your salary expectation?")

In [ ]:
# contents = [
#     types.Content(
#       role="user",
#       parts=[
#         types.Part.from_text(text="""Please introduce yourself""")
#       ]
#     )
#   ]

## Answer Questions

In [ ]:
# for chunk in client.models.generate_content_stream(
#     model = model_name,
#     contents = contents,
#     config = generate_content_config,
#     ):
#     print(chunk.text, end="")